In [1]:
import spacy
import pandas as pd
from spacy.util import minibatch, compounding
import random 

In [36]:
n_iter = 200
base_df = pd.read_csv('../datasets/TrainNER.csv', delimiter=';', encoding='cp1252')
base_df.describe()

,Sentence #,Word,POS,Tag
count,9000,196645,196645,196645
unique,9000,15323,41,15
top,Sentence: 435,the,NN,O
freq,1,9839,27417,166610


In [77]:
def add_SenteceID(df):
    for index, row in df.iterrows():
        if str(row['Sentence #']) != 'nan':
            current_sentence = int(row['Sentence #'].split(' ')[1])
        df.loc[index, 'SentenceID'] = current_sentence
    df.loc[len(df.index)-1, 'Sentence #'] = 'Sentence ' + str(df.loc[len(df.index) - 2, 'SentenceID'] + 1)
    return df

def get_sentence(id):
    words = base_df[(base_df['SentenceID'] == id)]['Word'].values.tolist()
    return ' '.join(str(word) for word in words)

def get_sentence_entities(sentence):
    entities = {'entities': None}
    entity = []
    for word in sentence.split(' '):
        entity.append((sentence.find(word), sentence.find(word)+len(word), str(base_df[base_df['Word'] == word]['Tag'].iloc[0])))
    entities['entities'] = entity
    return entities

def get_acc(df):
    tp = df.loc[(df['Tag'] == df['Prediction']) & (df['is_trained'] == False)].shape[0]
    return tp / (df[df['is_trained'] == False].shape[0]-1) * 100

def get_recall(df):
    tp = df.loc[(df['Tag'] != 'O') & (df['Tag'] == df['Prediction']) & (df['is_trained'] == False)].shape[0]
    tp_fn = df.loc[(df['Tag'] != 'O') & (df['is_trained'] == False)].shape[0] - 1
    return tp / tp_fn * 100

def get_precision(df):
    tp = df.loc[(df['Tag'] != 'O') & (df['Tag'] == df['Prediction']) & (df['is_trained'] == False)].shape[0]
    tp_fp = df.loc[(df['Prediction'] != 'O') & (df['is_trained'] == False)].shape[0] - 1
    return tp / tp_fp

In [17]:
# sentence_df = pd.DataFrame(columns=["Sentence", "Entities"])
# for i in base_df['SentenceID'].unique().tolist():
#     sentence = get_sentence(i)
#     entities = {'entities': None}
#     entity = []
#     try:
#         for word in sentence.split(' '):
#             entity.append((sentence.find(word), sentence.find(word)+len(word), str(base_df[base_df['Word'] == word]['Tag'].iloc[0])))
#         entities['entities'] = entity
#         sentence_df = sentence_df.append({'Sentence': sentence, 'Entities': entities}, ignore_index=True)
#     except:
#         print("unfurtunatelly an error has ocurred")

# sentence_df.to_csv('../datasets/SentenceEntities.csv')

In [18]:
# sentence_df.to_csv('../datasets/SentenceEntities.csv', sep=';')

In [60]:
def create_train_data(df, sample_num):
    rands = random.sample(df['Sentence #'].unique().tolist(), sample_num)
    rands = [int(x.split(' ')[1])+1 for x in rands]
    train_data = []
    counter = 0
    for num in rands:
        sent = get_sentence(num)
        entities = get_sentence_entities(sent)
        train_data.append((sent, entities))
        counter += 1
        if counter % 100 == 0:
            print(str(counter) + " training data have created")
    return train_data

In [20]:
# train.loc['entities'] = train.apply(lambda row: {'entities':[(0, len(str(row['Word'])), str(row['Tag']))]}, axis=1)
# train['Word'] = train['Word'].apply(str)
# subset = train[['Word', 'entities']]
# train_tuples = [tuple(x) for x in subset.values]

In [62]:
# base_df = add_SenteceID(base_df)
base_df.tail()

,Sentence #,Word,POS,Tag,SentenceID
196641,NaN,and,CC,O,9000.0
196642,NaN,foreign,JJ,O,9000.0
196643,NaN,investment,NN,O,9000.0
196644,NaN,.,.,O,9000.0
196645,Sentence 9001.0,NaN,NaN,NaN,9000.0


In [65]:
train_data = create_train_data(base_df, 1000)
# train_data

100training data have created
200training data have created
300training data have created
400training data have created
500training data have created
600training data have created
700training data have created
800training data have created
900training data have created
1000training data have created


[('It warns that the storm could strengthen again later Thursday .',
  {'entities': [(0, 2, 'O'),
    (3, 8, 'O'),
    (9, 13, 'O'),
    (14, 17, 'O'),
    (18, 23, 'O'),
    (24, 29, 'O'),
    (30, 40, 'O'),
    (41, 46, 'O'),
    (47, 52, 'O'),
    (53, 61, 'B-time'),
    (62, 63, 'O')]}),
 ('The U.S. envoy spoke after meeting with Syrian President Bashar al-Assad .',
  {'entities': [(0, 3, 'O'),
    (4, 8, 'B-gpe'),
    (9, 14, 'O'),
    (15, 20, 'O'),
    (21, 26, 'O'),
    (27, 34, 'O'),
    (35, 39, 'O'),
    (40, 46, 'B-gpe'),
    (47, 56, 'B-per'),
    (57, 63, 'I-per'),
    (64, 72, 'I-per'),
    (5, 6, 'O')]}),
 ('The broadcaster says Kate Peyton was rushed to a local hospital after the attack and underwent surgery .',
  {'entities': [(0, 3, 'O'),
    (4, 15, 'O'),
    (16, 20, 'O'),
    (21, 25, 'B-per'),
    (26, 32, 'I-per'),
    (33, 36, 'O'),
    (37, 43, 'O'),
    (29, 31, 'O'),
    (7, 8, 'O'),
    (49, 54, 'O'),
    (55, 63, 'O'),
    (64, 69, 'O'),
    (70, 73, 'O'),

In [67]:
n_iter = 300
nlp = spacy.blank('en')
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)

for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# for index, row in train.iterrows():
#     ner.add_label(str(row['Tag']))
    
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,
                annotations,
                drop=0.5,
                sgd=optimizer,
                losses=losses
            )
        print(itn, losses)

0 {'ner': 782.0909485168897}
1 {'ner': 512.635843603919}
2 {'ner': 434.1107023519703}
3 {'ner': 383.04548408022254}
4 {'ner': 351.6153027362848}
5 {'ner': 313.6018903113977}
6 {'ner': 295.3323630886571}
7 {'ner': 280.0426615623787}
8 {'ner': 260.02338179911686}
9 {'ner': 244.57406043769907}
10 {'ner': 236.49560117051408}
11 {'ner': 218.1295198275675}
12 {'ner': 208.2552369827217}
13 {'ner': 198.75453701231237}
14 {'ner': 180.30623828936064}
15 {'ner': 175.63587500939016}
16 {'ner': 162.15094466062973}
17 {'ner': 160.3682065761179}
18 {'ner': 153.3407425077249}
19 {'ner': 150.75394265004363}
20 {'ner': 137.34653671694264}
21 {'ner': 131.30596966698414}
22 {'ner': 132.51469280759378}
23 {'ner': 120.10188414971392}
24 {'ner': 123.88342333146332}
25 {'ner': 111.60355527651836}
26 {'ner': 106.11035285579872}
27 {'ner': 110.53746766372291}
28 {'ner': 106.1825611819177}
29 {'ner': 103.62520206950607}
30 {'ner': 98.59825169909405}
31 {'ner': 98.30147272712402}
32 {'ner': 91.11105136729135}
33 

261 {'ner': 14.531632163071956}
262 {'ner': 14.537508041932178}
263 {'ner': 14.620487760369281}
264 {'ner': 17.064623145655027}
265 {'ner': 16.346954033680323}
266 {'ner': 16.1531149388599}
267 {'ner': 15.07355907965234}
268 {'ner': 15.108567473458416}
269 {'ner': 16.315244319461065}
270 {'ner': 13.972864747433743}
271 {'ner': 15.027916614205242}
272 {'ner': 17.50260205056062}
273 {'ner': 15.38307234058713}
274 {'ner': 17.02008688000764}
275 {'ner': 13.596689536451365}
276 {'ner': 13.649045994887617}
277 {'ner': 15.552586161003502}
278 {'ner': 14.39462660978465}
279 {'ner': 17.002872364800517}
280 {'ner': 13.903173582225858}
281 {'ner': 15.65257012223047}
282 {'ner': 13.112499048330493}
283 {'ner': 14.006502220023103}
284 {'ner': 15.448981602068228}
285 {'ner': 12.909710652086348}
286 {'ner': 12.807890225666322}
287 {'ner': 14.232805513682456}
288 {'ner': 15.262544886790618}
289 {'ner': 16.066026520723803}
290 {'ner': 13.652101677749224}
291 {'ner': 16.177689401390506}
292 {'ner': 15.5

In [76]:
predicted_df = base_df
sentenceID = 1
doc = nlp(get_sentence(sentenceID))
is_train_data = False
for index, row in predicted_df.iterrows():
    if sentenceID != row['SentenceID']:
        is_train_data = False
        sentenceID = row['SentenceID']
        sentence = get_sentence(sentenceID)
        doc = nlp(sentence)
        for data in train_data:
            if sentence == data[0]:
                is_train_data = True
                break
    for ent in doc.ents:
        if row['Word'] == str(ent):
            predicted_df.loc[index,'Prediction'] = ent.label_
            break
    if str(predicted_df.loc[index,'Prediction']) == 'nan':
        predicted_df.loc[index,'Prediction'] = 'O'
    predicted_df.loc[index, 'is_trained'] = is_train_data
    if index % 10000 == 0:
        print(str(index) + ' rows predicted')

predicted_df.to_csv('../datasets/TrainingPrediction3.csv', sep=';')
predicted_df.tail()

0 rows predicted
10000 rows predicted
20000 rows predicted
30000 rows predicted
40000 rows predicted
50000 rows predicted
60000 rows predicted
70000 rows predicted
80000 rows predicted
90000 rows predicted
100000 rows predicted
110000 rows predicted
120000 rows predicted
130000 rows predicted
140000 rows predicted
150000 rows predicted
160000 rows predicted
170000 rows predicted
180000 rows predicted
190000 rows predicted


,Sentence #,Word,POS,Tag,SentenceID,Prediction,is_trained
196641,NaN,and,CC,O,9000.0,O,False
196642,NaN,foreign,JJ,O,9000.0,O,False
196643,NaN,investment,NN,O,9000.0,O,False
196644,NaN,.,.,O,9000.0,O,False
196645,Sentence 9001.0,NaN,NaN,NaN,9000.0,O,False


In [78]:
print("Acc: " + str(get_acc(predicted_df)))
print("Recall: " + str(get_recall(predicted_df)))
print("Precision: " + str(get_precision(predicted_df)))

Acc: 92.03240860030347
Recall: 55.38467323456418
Precision: 0.5677009057621892
